In [3]:
import GetPropertiesAPI as GP
from collections import Counter
import pandas as pd
import importlib
importlib.reload(GP)
import statistics as s
import operator
from collections import defaultdict
import csv 
import matplotlib.pyplot as plt

In [15]:
aid_list = []
gidAidDict = {}
for gid in range(1,9407):
    aid = GP.getAnnotID(gid)
    aid_list.append(aid)
    gidAidDict[gid] = aid
    
aid_list = list(filter(lambda x: x != None,aid_list))

In [16]:
aids = []
for aid_l in aid_list:
    for aid in aid_l:
        aids.append(aid)

In [20]:
aidContribTupList = []
for aid in aids:
    contrib = GP.getImageFeature(aid,'image_contributor_tag')
    aidContribTupList.append((aid,contrib[0]))

In [21]:
aidNidMap = {}
aidNamesMap = {}
aidNidTupList = [] # modified

for aid in aids:
    nid = GP.getImageFeature(aid,'nids')
    aidNidMap[aid] = nid
    aidNidTupList.append((aid,nid[0]))

In [22]:
nids = []
for aid in aidNidMap.keys():
    nids.append(aidNidMap[aid][0])

In [23]:
nids = list(filter(lambda x : x > 0,nids))

In [9]:
counter_nid= Counter(nids)


In [10]:
gidAidDict
aidGidTupList = [] # key : aid and value : gid # modified
for gid in gidAidDict.keys():
    if gidAidDict[gid] != None:
        for aid in gidAidDict[gid]:
            aidGidTupList.append((aid,gid))

In [11]:
aidGidDf = pd.DataFrame(aidGidTupList,columns = ['AID','GID']) 
aidNidDf = pd.DataFrame(aidNidTupList,columns = ['AID','NID']) 
aidContribDf = pd.DataFrame(aidContribTupList,columns = ['AID','CONTRIBUTOR'])
aidNidDf = aidNidDf[(aidNidDf['NID']>0)]

In [12]:
aidGidNidDf = pd.merge(aidGidDf,aidNidDf,left_on = 'AID',right_on = 'AID')
aidGidNidContribDf = pd.merge(aidGidNidDf,aidContribDf,left_on = 'AID',right_on = 'AID')

In [13]:
aidGidNidContribDf.to_csv('results.csv',index=False)

In [157]:
with open('results.csv') as f: # read from csv file into a key : GID and value : CONTRIBUTOR
    reader = csv.DictReader(f)
    gidContribMap = { line['GID']: line['CONTRIBUTOR'] for line in reader }
        
len(gidContribMap)

4948

In [277]:
ContribTotal = {} # dict with key : CONTRIBUTOR and value: Total photos taken

for gid,contrib in gidContribMap.items():
     ContribTotal[contrib] = ContribTotal.get(contrib,0) + 1

print(s.mean(ContribTotal.values()))
print(s.stdev(ContribTotal.values()))

88.35714285714286
136.7916037523607


In [3]:
with open('results.csv') as f2: # read from csv file into a Dict with key : AID and value : GID, NID, CONTRIBUTOR
    reader2 = csv.DictReader(f2)
    aidToGidNidContribMap = { line['AID']: [line['GID'], line['NID'], line['CONTRIBUTOR']] for line in reader2 }

In [275]:
NidContribTotal = {} # dict with key : NID, CONTRIBUTOR and value: Total photos taken
for aid,(gid,nid,contrib) in aidToGidNidContribMap.items():
     NidContribTotal[nid,contrib] = NidContribTotal.get((nid,contrib),0) + 1

In [234]:
csv_out = csv.writer(open('nidtoContributor.csv', 'w')) # rename file, results of Nid to Contributor to Total
csv_out.writerow(['NID', 'CONTRIBUTOR', 'TOTAL'])
for (Nid, Contrib), value in NidContribTotal.items():
   csv_out.writerow([Nid, Contrib, value])

In [272]:
#from collections import defaultdict

averageCountofPictures = defaultdict(list) # dict where key : NID and values: list of pictures taken per photographer

for (nid, contrib), total in NidContribTotal.items(): 
    averageCountofPictures[nid].append(total)
       
#averageCountofPictures

In [276]:
countUniquePhotoPerPic = {} # dict where key : NID and values : # of CONTRIBUTERS

for (nid, contrib), total in NidContribTotal.items():
    countUniquePhotoPerPic[nid] = countUniquePhotoPerPic.get((nid),0) + 1 

#countUniquePhotoPerPic['741']

In [ ]:
#JUST LOOK FROM HERE

In [60]:
# Arguments : Required Feature
# Accepted Features: species_texts, age_months_est, exemplar_flags, sex_texts, yaw_texts, quality_texts,image_contributor_tag
# Returns : Returns Dictionary of total feature
def getContributorFeature(feature):
    
    #SHOULD WE HAVE THIS????
    with open('results.csv') as f: # read from csv file into a Dict with key : AID and value : GID, NID, CONTRIBUTOR
        reader = csv.DictReader(f)
        aidToGidNidContribMap = { line['AID']: [line['GID'], line['NID'], line['CONTRIBUTOR']] for line in reader }
    
    
    contribToFeatureMap = defaultdict(list) # dict where key : contributor and values : List of feature
    for aid,(gid,nid,contrib) in aidToGidNidContribMap.items():
        contribToFeatureMap[contrib].append(GP.getImageFeature(aid, feature)[0])
    
    contribAnimFeatCount = {} # dict where key : contributor and values : total of specific feature
    for key in contribToSexMap.keys():
        contribAnimFeatCount[key]=dict(Counter(contribToFeatureMap[key]))
    
    return contribAnimFeatCount

In [ ]:
#m={}
#x={}
#m=getContributorFeature("species_texts")
#x=getContributorFeature("sex_texts")
#print(m)

In [79]:
#FOR ALL MALES, FEMALES, UNKNOWN INTO CSV FILE

malesTotal={}
femaleTotal={}
unknownTotal={}

for contrib, feature in x.items(): #change x
    malesTotal[contrib]=feature.get('Male', 0)
    femaleTotal[contrib]=feature.get('Female', 0)
    unknownTotal[contrib]=feature.get('UNKNOWN SEX', 0)
    
maleTotal=(sum(malesTotal.values()))
femaleTotal=(sum(femaleTotal.values()))
unknownTotal=(sum(unknownTotal.values()))

csv_out = csv.writer(open('contribSexTotal.csv', 'w')) # rename file, results of Nid to Contributor to Total
csv_out.writerow(['FEATURE', 'MALE', 'FEMALE', 'UNKWOWN SEX'])
csv_out.writerow(['sex_texts', malesTotal, femaleTotal, unknownTotal ])

2505

In [68]:
# Arguments : ContributorToFeatureDict , Required Specific Feature
# Accepted Specific Features: sex_texts = "Male", "Female", "UNKNOWN SEX", etc.
# Returns : 
def getContributorSpecificFeature(contribAnimFeatCount, specificfeat):
    contribSpecFeatureMap={}
    for contrib, feature in contribAnimFeatCount.items():
        contribSpecFeatureMap[contrib]=feature.get(specificfeat , 0)
    
    csv_out = csv.writer(open('contrib'+ specificfeat +'Map.csv', 'w')) #used for plotting later
    csv_out.writerow(['CONTRIBUTOR', specificfeat])
    for contrib, specfeature in contribSpecFeatureMap.items():
       csv_out.writerow([contrib, specfeature])

In [69]:

#getContributorSpecificFeature(contribAnimSexCount, "UNKNOWN SEX")
getContributorSpecificFeature(contribAnimSexCount, 'Female')


In [4]:
# Arguments : csv_file , Required Specific Feature
# Accepted Specific Features: sex_texts = "Male", "Female", "UNKNOWN SEX", etc.
# Returns : NONE
def creategraph(csv_file, specific_feature):
    data = pd.read_csv(csv_file, sep=',',header=0, index_col =0) #csv_file

    data.plot(kind='bar')
    plt.ylabel('Number of ' + specific_feature + ' taken')
    plt.xlabel('Contributor')
    plt.title('Contributor to'+ specific_feature + 'Totals')

    plt.show()

In [ ]:
#creategraph('contribMaleMap.csv', "Male")
creategraph('contribFemaleMap.csv', 'Female')